In [ ]:
# !pip install tokenizers
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.7 MB/s eta 0:00:00


In [227]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from tokenizers import Tokenizer,PreTokenizedString,NormalizedString, pre_tokenizers, CharBPETokenizer , decoders,  normalizers
from tokenizers.models import BPE , WordPiece
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace, PreTokenizer
from tokenizers.processors import TemplateProcessing
from tokenizers.normalizers import NFD, Lowercase, StripAccents
torch.manual_seed(45562)

# Read the dataset

In [228]:
text = open('input.txt', 'r', encoding='utf-8').read()
print("length of dataset in characters: ", len(text))
# print(f"\n{text[:200]}")

length of dataset in characters:  1115394


# Build a custom tokenizer with hugging face  tokenizer API

In [ ]:
pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Split("\n",'merged_with_previous'),Whitespace()])

In [ ]:
pre_tokenizer.pre_tokenize_str(text[:50])

[('First Citizen:\n', (0, 15)),
 ('Before we proceed any further, hear', (15, 50))]

In [288]:
class CustomPreTokenizer:
    "A pre_tokenizer that preserves the preserves the new line character"
    def line_split(self, i: int, normalized_string: NormalizedString) -> List[NormalizedString]:
        splits = []
        for token in normalized_string.split(" "):
          if "\n" in token:
              subtokens = token.split("\n")
              for i, subtoken in enumerate(subtokens):
                  if i > 0:
                      splits.append("\n")
                  if subtoken:
                      splits.append(subtoken)
          else:
              splits.append(token)
        return [NormalizedString(split).split(behavior="N") for split in splits]

    # def pre_tokenize(self, pretok):
    #     pretok.split(self.line_split)#.line_split(pretok)

    def pre_tokenize(self, pretok: PreTokenizedString):
        pretok.split(self.line_split)

In [261]:
a

['First',
 'Citizen:',
 '\n',
 'Before',
 'we',
 'proceed',
 'any',
 'further,',
 'hear']

In [289]:
tok = Tokenizer(BPE())
tok.normalizer = normalizers.Sequence([NFD(), StripAccents()])
tok.pre_tokenizer = PreTokenizer.custom(CustomPreTokenizer())
# tok.decoder = Decoder.custom(CustomDecoder())
print(tok.pre_tokenizer.pre_tokenize_str(text[:50]))

Exception: ignored

In [267]:
sp_tokenizer = Tokenizer(BPE(unk_token="<unk>"))
sp_tokenizer.normalizer = normalizers.Sequence([NFD(), StripAccents()])
sp_tokenizer.pre_tokenizer = PreTokenizer.custom(CustomPreTokenizer())
sp_tokenizer.decoder = decoders.BPEDecoder(suffix="</w>")
class CustomDecoder:
    def decode(self, tokens: List[str]) -> str:
        return "".join(tokens)
# sp_tokenizer.post_processor = TemplateProcessing(single="[CLS] $A [SEP]",
#                                                  pair="[CLS] $A [SEP] $B:1 [SEP]:1",
#                                                  special_tokens=[
#                                                     ("[CLS]", 1),
#                                                     ("[SEP]", 2),
#                                                     ("[NEW-LINE]",3)])

# trainer = BpeTrainer(vocab_size=500, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
# sp_tokenizer.train_from_iterator(text , trainer)

In [270]:
dir(PreTokenizer.custom)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__text_signature__']

In [268]:
print(sp_tokenizer.pre_tokenizer.pre_tokenize_str("Hello"))

Exception: ignored

In [ ]:
toe = CharBPETokenizer()

In [ ]:
tokenizer = Tokenizer.from_pretrained("bert-base-uncased")

In [ ]:
sp_tokenizer.save("sp-tokenizer.json")

In [ ]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer.pre_tokenizer = Whitespace()
tokenizer.train_from_iterator(text, trainer)
tokenizer.save("sp-tokenizer.json")

In [ ]:
output = tokenizer.encode(text[:50])
output.tokens

['[CLS]',
 'first',
 'citizen',
 ':',
 'before',
 'we',
 'proceed',
 'any',
 'further',
 ',',
 'hear',
 '[SEP]']

In [ ]:
import jieba

from typing import List

from tokenizers import Tokenizer, Regex, NormalizedString, PreTokenizedString
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import PreTokenizer
from tokenizers.normalizers import Normalizer
from tokenizers.decoders import Decoder


class JiebaPreTokenizer:
    def jieba_split(self, i: int, normalized_string: NormalizedString) -> List[NormalizedString]:
        splits = []
        # we need to call `str(normalized_string)` because jieba expects a str,
        # not a NormalizedString
        for token in normalized_string:
            splits.append(token)

        return splits
        # We can also easily do it in one line:
        # return [normalized_string[w[1] : w[2]] for w in jieba.tokenize(str(normalized_string))]

    # def odd_number_split( self, i: int, normalized_string: NormalizedString) -> List[NormalizedString]:
    #     # Just an odd example...
    #     splits = []
    #     last = 0
    #     for (i, char) in enumerate(str(normalized_string)):
    #         if char.isnumeric() and int(char) % 2 == 1:
    #             splits.append(normalized_string[last:i])
    #             last = i
    #     # Don't forget the last one
    #     splits.append(normalized_string[last:])
    #     return splits

    def pre_tokenize(self, pretok: PreTokenizedString):
        # Let's call split on the PreTokenizedString to split using `self.jieba_split`
        pretok.split(self.jieba_split)
        # Here we can call `pretok.split` multiple times if we want to apply
        # different algorithm, but we generally just need to call it once.
        # pretok.split(self.odd_number_split)


class CustomDecoder:
    def decode(self, tokens: List[str]) -> str:
        return "".join(tokens)


class CustomNormalizer:
    def normalize(self, normalized: NormalizedString):
        # Most of these can be replaced by a `Sequence` combining some provided Normalizer,
        # (ie Sequence([ NFKC(), Replace(Regex("\s+"), " "), Lowercase() ])
        # and it should be the prefered way. That being said, here is an example of the kind
        # of things that can be done here:
        normalized.nfkc()
        normalized.filter(lambda char: not char.isnumeric())
        normalized.replace(Regex("\s+"), " ")
        normalized.lowercase()


# This section shows how to attach these custom components to the Tokenizer
tok = Tokenizer(BPE())
tok.normalizer = Normalizer.custom(CustomNormalizer())
tok.pre_tokenizer = PreTokenizer.custom(JiebaPreTokenizer())
tok.decoder = Decoder.custom(CustomDecoder())

input = "永和服装饰品有限公司"
print("PreTokenize:", input)
print(tok.pre_tokenizer.pre_tokenize_str(input))
# [('永和', (0, 2)), ('服装', (2, 4)), ('饰品', (4, 6)), ('有限公司', (6, 10))]

input = "112233"
print("PreTokenize:", input)
print(tok.pre_tokenizer.pre_tokenize_str(input))
# [('1', (0, 1)), ('122', (1, 4)), ('3', (4, 5)), ('3', (5, 6))]

input = "1234 ℌ𝔢𝔩𝔩𝔬    𝔱𝔥𝔢𝔯𝔢 𝓂𝓎 𝒹ℯ𝒶𝓇 𝕕𝕖𝕒𝕣    𝕗𝕣𝕚𝕖𝕟𝕕!"
print("Normalize:", input)
print(tok.normalizer.normalize_str(input))
# " hello there my dear dear friend!"

PreTokenize: 永和服装饰品有限公司


In [275]:
list(jieba.tokenize(str("永和服装饰品有限公司")))

[('永和', 0, 2), ('服装', 2, 4), ('饰品', 4, 6), ('有限公司', 6, 10)]